# 天体力学第一次上机
我们需要完成以下任务：
1. 站心、地心、日心坐标系的转换 
2. 平太阳时和平恒星时的转换

> 时角和赤经方向相反

In [1]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import datetime
import time
%matplotlib inline

In [77]:
def UTC2SG(yyyy,mm,dd,hours):
    # J2000，从UTC计算对应的jd_目的是最终算出儒略世纪数
    J2000 = 2451545.0
    # J2000：UTC时间2000年1月1日11:58:55.816。
    date_start = datetime.datetime.strptime("2000-1-1 11:58:55","%Y-%m-%d %H:%M:%S")
    Date_str = str(yyyy) +'-'+ str(mm) +'-'+ str(dd)
    Date = datetime.datetime.strptime(Date_str,"%Y-%m-%d")
    
    # 用unix时间戳计算到J2000的时间差
    un_start = time.mktime(date_start.timetuple()) + 0.816
    un_this = time.mktime(Date.timetuple()) + hours*3600
    
    # 返回儒略世纪数（1儒略年=31557600 SI秒）
    t = (un_this - un_start)/(31557600*100)
    # 给出平恒星时(s为单位)
    SG = 18.6973746*3600 + 879000.0513367*3600*t + 0.093104*t**2 - 6.2e-6*t**3
    # degree = SG%(3600*24)/(3600*24)*360
    return SG,t

In [78]:
# example，得出的SG去整除每天的秒数就可以得到当天的恒星时，最后再由恒星时算出相应的角度
SG,t = UTC2SG(1999,3,22,6)
degree = SG%(3600*24)/(3600*24)*360
degree

269.57287335637955

In [79]:
## 坐标系转换
# 从 A 到 B， 以下程序中 P_I 即 相对于I坐标系P的位置。同理 A_B 指的是在 B 坐标系中 A 的位置。P_B = A_B + P_A
# 本次我们需要计算人造卫星以及小行星的初轨:
# 对于人造卫星，我们需要用到 站心-地心 的转换。
# 对于小行星，我们需要用到 站心-地心-日心，最后在日心：赤道-黄道